In [5]:
# Imports
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import pickle

# Import Our environment
from dev_env import tradingEng

In [6]:
# Define environment
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)

In [ ]:
env = tradingEng(paths, action = 'small', obs = 'xs')

# Instantiate the agent
model = PPO("MlpPolicy", env, verbose=True, batch_size = 1024)

# Train the agent and display a progress bar
model.learn(total_timesteps=int(1e7), progress_bar=True)

# Save the agent
model.save("PPO")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.8     |
|    ep_rew_mean     | -15.4    |
| time/              |          |
|    fps             | 1375     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
